In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import urllib.request
import json
import csv
import sys
import time

In [2]:
'data/math_query.csv'

class WikidataConverter():
    """cleans a csv file of wikidata nodes and finds 
       english wikipedia articles, if available.
    """
    def __init__(self):
        self.total_written = 0
        self.total_skipped = 0
    
    def parse(self, input_file, output_file): 
        
        currently_written = 0
       
        try:
            with open(str(output_file), 'r') as f:
                reader = csv.reader(f, delimiter=',')
                next(reader, None)
                existing_ids = {i[0]: i[1] for i in reader}
        except:
            with open(str(output_file), 'w') as f:
                writer = csv.writer(f, delimiter=',')
                writer.writerow(['wikidata_id', 'title', 'wikipedia_url', 'wikidata_url'])
                existing_ids = {}
        
        self.total_written = len(existing_ids)
        
        count = 0   
        with open(str(input_file), 'r') as f:
            reader = csv.reader(f, delimiter=',')
            count = sum(1 for row in f)
            
        with open(str(input_file), 'r') as infile, open(str(output_file), 'a') as outfile:
            wikidata_base = 'http://www.wikidata.org/entity/'
            wikipedia_base = 'https://en.wikipedia.org/wiki/'
            base_len = len(wikidata_base)
            reader = csv.reader(infile, delimiter=',')
            writer = csv.writer(outfile, delimiter=',')
            next(reader, None) 
            i = 0
            while i < count:
                row = next(reader)
                data_url = row[0]
                data_id = data_url[base_len:]
                if data_id in existing_ids:
                    continue
                else:
                    title = row[1]
                    existing_ids[data_id] = title
                    title = title.replace(' ', '_')
                    wikipedia_url = wikipedia_base + title
                    try:
                        html = urllib.request.urlopen(wikipedia_url).read()
                        writer.writerow([data_id, title, wikipedia_url, data_url])
                        self.total_written += 1
                        currently_written += 1
                        percent = round(i/count, 4)
                        sys.stdout.write('\r'+ 'percent_complete: ' + str(percent) + ' currently_written: ' + str(currently_written) + ' total written: ' \
                                         + str(self.total_written))
                    except:
                        self.total_skipped +=1
                        percent = round(i/count, 4)
                        sys.stdout.write('\r'+ 'percent_complete: ' + str(percent) + ' currently_written: ' + str(currently_written) + ' total written: ' \
                                         + str(self.total_written))
                i += 1

In [3]:
parse = WikidataConverter()

In [ ]:
parse.parse('data/academics/query_academics.csv', 'data/academics/academics_en_links.csv')

percent_complete: 0.0004 currently_written: 55 total written: 55

In [ ]:
parse.total_skipped

In [12]:
str(round(0.522, 2))

'0.52'

In [15]:
import os
import time
from selenium.webdriver import Firefox
from bs4 import BeautifulSoup as soup

def get_expand_buttons(browser):
    """Return a list of expand buttons to click on."""
    return browser.find_elements_by_xpath("//span[@title='expand']")

total_links = []

def expand_all_categories(browser):
    """Expand all categories on the page."""
    expand_buttons = get_expand_buttons(browser)
    global total_links
    total_links = []
    i = 0
    while i<6:
        for button in expand_buttons:
            if button.is_displayed():
                button.click()
                time.sleep(0)
        lst = []
        for a in browser.find_elements_by_xpath('.//a'):
            lst.append(a.get_attribute('href'))
        total_links.append(lst)
        expand_buttons = get_expand_buttons(browser)
        
        i += 1
        
def main():
    """Open the page and expand all categories."""
    browser = Firefox()
    browser.get('https://en.wikipedia.org/wiki/Special:CategoryTree?target=mathematics&mode=all&namespaces=&title=Special%3ACategoryTree')
    expand_all_categories(browser)

In [ ]:
browser = Firefox()
browser.get('https://en.wikipedia.org/wiki/Special:CategoryTree?target=mathematics&mode=all&namespaces=&title=Special%3ACategoryTree')
expand_all_categories(browser)

In [ ]:
for links in total_links:
    print(len(links))

In [10]:
lst = []
for a in browser.find_elements_by_xpath('.//a'):
    lst.append(a.get_attribute('href'))

In [ ]:
len(lst)